## ONNX Runtime 으로 Lenet-1 모델 돌리기
- lenet-1_tanh.onnx 모델 로드 및 Inference 진행
- mnist Data 이용, Train, Test 진행
- 사전 학습 완료된 파일 : lenet-1_tanh.onnx
    

In [1]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras import utils
from PIL import Image

# image resize 함수
def img_resize(value, img_array):
    print("--images are being resizing--")
    result = np.zeros((len(img_array), value[0], value[1]))

    for index in range(len(img_array)):
        img = Image.fromarray(img_array[index], 'L')
        img = img.resize((value[0], value[1]))
        img = np.array(img)
        result[index] = img
    print("--image resize complete--")
    return result

# mnist Data Load 및
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

input_shape = (28, 28)

# data resize
X_train = img_resize(input_shape, X_train)
X_test = img_resize(input_shape, X_test)
print('data resize ',X_train.shape, X_test.shape)

# 차원 확장
X_train = np.expand_dims(X_train, axis = -1)
X_test = np.expand_dims(X_test, axis = -1)
print('expand_dims ',X_train.shape, X_test.shape)

# 정답값 categorical로
Y_train = utils.to_categorical(Y_train, 10)
Y_test = utils.to_categorical(Y_test, 10)
print('to_categorical ',X_train.shape, X_test.shape)

print('전처리 작업 완료')

--images are being resizing--
--image resize complete--
--images are being resizing--
--image resize complete--
data resize  (60000, 28, 28) (10000, 28, 28)
expand_dims  (60000, 28, 28, 1) (10000, 28, 28, 1)
to_categorical  (60000, 28, 28, 1) (10000, 28, 28, 1)
전처리 작업 완료


In [11]:
# ONNX Runtime 완료
import onnxruntime as rt
sess = rt.InferenceSession("../model/lenet-1_tanh.onnx")

input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

# sess.run
pred_onx = sess.run([label_name], {input_name: X_test.astype(np.float32)})[0]

print('---------- y_pred ----------')
print(pred_onx)
print('---------- y_test ----------')
print(Y_test)

---------- y_pred ----------
[[0.06258683 0.05771666 0.03960286 ... 0.3636641  0.05365551 0.12282629]
 [0.0621072  0.1272365  0.11648028 ... 0.03695392 0.09034108 0.04458275]
 [0.08137407 0.23129898 0.14329015 ... 0.0712693  0.07874458 0.10569439]
 ...
 [0.05942455 0.06414514 0.06736682 ... 0.16207992 0.1659305  0.08778336]
 [0.06988104 0.08422282 0.08181541 ... 0.07119451 0.17657767 0.11906472]
 [0.1480164  0.06119221 0.128175   ... 0.08134987 0.03393915 0.14120178]]
---------- y_test ----------
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [9]:
# 정확도 보기
from sklearn.metrics import accuracy_score

# 테스트 predict 결과들 비교 (평가지표 보기위함)
pred = np.round(np.array(pred_onx).flatten().tolist())
test = np.array(np.array(Y_test).flatten().tolist())

k_accuracy = float(accuracy_score(test, pred))
print(k_accuracy)

0.90006
